In [1]:
# import required packages
import requests
import ast # abstract syntax tree
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# get brewery names and locations

# connect to URL
URL= 'https://www.brewbound.com/breweries'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

# get rquired html - list of breweries is nested within map script
data = soup.find_all('script')[-4]
string_results = str(data).split("\r\n")[2]
string_results_list = "["+string_results[:len(string_results) - 7]+"]"
results = ast.literal_eval(string_results_list)

# convert list of lists to df
df = pd.DataFrame(results, columns = ['brewery_name', 'latitude','longitude','url'])
df.head()
# save as csv
# df.to_csv('breweries_2022.csv')

,brewery_name,latitude,longitude,url
0,Faust Hotel Restaurant and Brew PUB,29.701406,-98.123061,/breweries/Faust_Hotel_Restaurant_and_Brew_PUB
1,Wolf Pack Brewing Company,44.662053,-111.099461,/breweries/Wolf_Pack_Brewing_Co
2,Defiant Brewing Company,41.058405,-74.022847,/breweries/Defiant_Brewing_Co
3,El Toro Brewing Company,37.146552,-121.621987,/breweries/El_Toro_Brewing_Co
4,Sebago Brewing Company,43.679212,-70.396424,/breweries/Sebago_Brewing_Co


In [ ]:
# use brewery url to get beers

# make sure beer url is not cut short with ...
pd.set_option('display.max_colwidth', None)

# initialize string for results
beers = []

# iterate through all rows of df
for i in range(len(df['url'])):
# for i in range(10): # used for testing
    # get url
    url = df['url'].iloc[[i]].to_string(header=False, index=False)
    # print(i) # use if desired to monitor progress
    full_url = "https://www.brewbound.com"+url
    # connect to url - try statement used to avoid connection errors/ missing pages
    try:
        page = requests.get(full_url, allow_redirects=False)
        soup = BeautifulSoup(page.content, "html.parser")
        # find brewery location
        addresses = soup.find_all("address")
        # find beers table
        results = soup.find(id="beers")
    except:
        results = None
    
    # get address/ address for breweries with more than one location
    addr = []
    for address in addresses:
        addr.append(address.text.strip())

    # get beer table headers and add other required headers
    headers = []
    try:
        for j in results.find_all("th"):
            title = j.text
            headers.append(title)
        result_headers = headers[0:2]
        result_headers.append('beer_url')
        result_headers.append('beer_url2')
        result_headers.append('url')
        result_headers.append('location')
        
    except AttributeError:
        continue
    
    # get contents of beer table
    data = []
    
    try:
        rows = results.find_all('tr')
        for row in rows:
            # get beer name, style and url
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            cols = [ele for ele in cols if ele] # Get rid of empty values
            cols = [ele for ele in cols if ele != "No records to display."] # Get rid of extra values
            urls = row.find_all('a', href=True)
            urls = [url['href'] for url in urls]
            all_cols = cols + urls
            data.append(all_cols)

        # add url and address to data - remove empty lists
        table = list(filter(None, data))
        table = [x.append(url) or x for x in table]
        table = [x.append(addr) or x for x in table]
        beers= beers + table
    except AttributeError:
        continue

# convert list of lists to df
df_beers = pd.DataFrame(beers, columns = result_headers)
df_beers = df_beers.drop(columns = 'beer_url2')
df_beers.head()
# save as csv
# df_beers.to_csv('beers_2022.csv')

In [ ]:
# use beer url to get more beer info

# make sure beer url is not cut short with ...
pd.set_option('display.max_colwidth', None)

# df_beers= pd.read_csv('beers_2022.csv') # use if running scrapes separately

beers_info = []

# iterate through beer_urls
for i in range(len(df_beers['beer_url'])):
# for i in range(500,1000): # used for testing
    url = df_beers['beer_url'].iloc[[i]].to_string(header=False, index=False)
    # print(i) # use if desired to monitor progress
    beer_url =  "https://www.brewbound.com"+url
    try:
        # get html that contains description/ABV/IBU info
        page = requests.get(beer_url)
        soup = BeautifulSoup(page.content, "html.parser")
        rows = soup.find_all('div', {"class": "col-sm-12", "id": "brandtext"})
    except:
        rows = None

    info = []
    headers = []
    
    try:
        for row in rows:
            # get column headers
            strong = row.find_all("strong")
            strong = [x.text for x in strong]
            row = row.text
            row = row.split("\n")
            row = [x.strip() for x in row]
            headers.append(strong)
            info.append(row)

        # info[0] contains description, info[1] contains availability, ibu, abv etc.
        # join all sentences in description to single string
        description = info[0]
        description.remove('Official description from the brewery:')
        description = ' '.join(description).strip()
        info = [description] +info[1]
        
        # flatten headers into single list
        headers = [item for sublist in headers for item in sublist]

        # remove empty strings unless correspond to missing data  
        clean_info = []
        for i in range(len(info)):
            if info[i] =='':
                if info[i-1] in headers:
                    clean_info.append(info[i])
                else:
                    continue
            else:
                clean_info.append(info[i])

        # remove headers from list and add url to data so join possible
        results = [x for x in clean_info if x not in headers]
        results.append(url)
        
        beers_info.append(results)
    except:
        continue

# add url column header
headers.append('beer_url')

# convert list of lists to dataframe
df_beers_info = pd.DataFrame(beers_info, columns= headers)
df_beers_info.head()

# save as csv
# df_beers_info.to_csv('beersinfo_2022.csv')

In [49]:
# Dictionary to map state to abbreviation in clean up below
# from https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [97]:
# Clean and join datasets
# df_beers= pd.read_csv('beers_2022.csv') # use this if running each scrape separately

# Clean up location string and split to city and state/country
df_beers['location'] = df_beers['location'].dropna().apply(lambda x: x.replace("\'","").replace("[","").replace("]","").replace("\"","").replace("\n"," "))
df_beers[['city', 'state_country']] = df_beers['location'].str.split(',',n=1, expand=True)
df_beers['city'] = df_beers['city'].dropna().apply(lambda x: x.strip())
df_beers['state_country'] = df_beers['state_country'].dropna().apply(lambda x: x.replace("USA","").replace("U.S.A.","").replace("U.S.A","")
                            .replace("United States","").replace("Ontario","").replace(",","").replace("Orange County","CA").replace("Maui","").replace("West","")
                            .replace("FRANCE","France").replace("Island","OH"))
df_beers['state_country'] = df_beers['state_country'].dropna().apply(lambda x: x.strip())
df_beers['state_country'] = df_beers['state_country'].map(us_state_to_abbrev).fillna(df_beers['state_country'])


# add location info to brewery df
df_brewery = pd.read_csv('breweries_2022.csv')
df_brewery = df_brewery.rename(columns={"Unnamed: 0": "brewery_id"})
df_brewery = pd.merge(df_brewery, df_beers[['url','state_country','city']], on='url',how='left').drop_duplicates()

# add brewery id to beers df
df_beers = pd.merge(df_beers,df_brewery[['url','brewery_id']],on='url',how='left')

# drop url from brewery
df_brewery = df_brewery.drop(columns=['url'])

df_beers_info = df_beers_info.rename(columns={"Official description from the brewery:": "description"})
df_beers = pd.merge(df_beers,df_beers_info[['description','Availability','ABV','IBU','beer_url']],on='beer_url',how='left')
df_beers = df_beers.rename(columns={"Unnamed: 0": "beer_id"})
df_beers = df_beers.drop(columns = ['beer_url','url','location','city','state_country'])

# df_beers[df_beers['brewery_id'].isnull()]
df_brewery.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5539 entries, 0 to 17766
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   brewery_id     5539 non-null   int64  
 1   brewery_name   5539 non-null   object 
 2   latitude       5539 non-null   float64
 3   longitude      5539 non-null   float64
 4   state_country  1435 non-null   object 
 5   city           1443 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 302.9+ KB


In [110]:
# given only 26% of breweries gave beer information - and thus city/state. Decided to try reverse geocoding with geopy
import time
from geopy.geocoders import Nominatim
  
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

state = []
country = []

for i in range(df_brewery.shape[0]):
# for i in range(12): # used for testing
    # get latitude and longitude as string
    Latitude = df_brewery['latitude'].iloc[[i]].to_string(header=False, index=False)
    Longitude = df_brewery['longitude'].iloc[[i]].to_string(header=False, index=False)

    # Nominatime api will only permit one API request per second so sleep timer used to ensure rate does not exceed limit
    time.sleep(1)
    try:
        # reverse geocode 
        location = geolocator.reverse(Latitude+","+Longitude)
        address = location.raw['address']
    
        # extract state and country
        state_i = address.get('state', '')
        state.append(state_i)
        country_i = address.get('country', '')
        country.append(country_i)
    except:
        # if request fails add empty string to list
        state.append("")
        country.append("")

# add data to brewery data frame
df_brewery['state_geo']= state
df_brewery['country_geo']= country

# check for nulls in reverse geocoded columns
df_brewery.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5539 entries, 0 to 17766
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   brewery_id     5539 non-null   int64  
 1   brewery_name   5539 non-null   object 
 2   latitude       5539 non-null   float64
 3   longitude      5539 non-null   float64
 4   state_country  1435 non-null   object 
 5   city           1443 non-null   object 
 6   state_geo      5539 non-null   object 
 7   country_geo    5539 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 389.5+ KB
None


,brewery_id,brewery_name,latitude,longitude,state_country,city,state_geo,country_geo
0,0,Faust Hotel Restaurant and Brew PUB,29.701406,-98.123061,NaN,NaN,Texas,United States
1,1,Wolf Pack Brewing Company,44.662053,-111.099461,NaN,NaN,Montana,United States
2,2,Defiant Brewing Company,41.058405,-74.022847,NY,Pearl River,New York,United States
39,3,El Toro Brewing Company,37.146552,-121.621987,CA,Morgan Hill,California,United States
59,4,Sebago Brewing Company,43.679212,-70.396424,NaN,NaN,Maine,United States


In [112]:
# save dataframes as sheets in xlsx file for use in Tableau
with pd.ExcelWriter("beers_breweries2022.xlsx", engine = 'xlsxwriter') as writer:
    df_brewery.to_excel(writer, sheet_name="breweries")  
    df_beers.to_excel(writer, sheet_name="beers") 
